# Utilizar IPython widgets (parte2)

En este notebook utilizamos widgets para filtrar y explorar un DataFrame <br>
Es la segunda parte del artículo sobre widgets interactivos para Jupyter Notebook: https://estrategiastrading.com/widgets-interactivos-en-python-ipywidgets/

En este ejemplo utilizo los datos de los cierres mensuales del SP500 entre 1970 y 2019.
## 1.Cargar librerías, paquetes y data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set()
import ipywidgets as widgets

In [2]:
file = 'SP500_Mensual.csv'
df = pd.read_csv(file,index_col="Date", parse_dates = True)
df = df.dropna()
df.tail()

,Close,LogRet,Year,Month
Date,,,,
2019-08-30,2926.459961,-0.018257,2019,8
2019-09-30,2976.739990,0.017035,2019,9
2019-10-31,3037.560059,0.020226,2019,10
2019-11-29,3140.979980,0.033480,2019,11
2019-12-31,3230.780029,0.028189,2019,12


## 2. Utilizando interact

In [3]:
# utilizando interact 
from ipywidgets import interact

interact autogenera controles para los argumentos de la función, y luego llama a la función con esos argumentos cuando manipulamos los controles de forma interactiva. <br>Para usar interact, necesitas definir una función que quieras explorar

### 2.1 Estadísticas por columna

In [4]:
#En este caso con Interact como decorador que es la manera más sencilla de crear un widget
@interact
def describe(column=list(df.columns)):
    print(df[column].describe())

interactive(children=(Dropdown(description='column', options=('Close', 'LogRet', 'Year', 'Month'), value='Clos…

### 2.2 Filtrar por filas según los valores del widget

In [5]:
# Con Interact como decorador para crear un widget
@interact
def mostrar_valores(
    Column =['LogRet'],
    x=(-0.25,0.25,0.05)):
    display(df.loc[df[Column] >= x])

interactive(children=(Dropdown(description='Column', options=('LogRet',), value='LogRet'), FloatSlider(value=0…

Como se ve en este ejemplo, se puede seleccionar el valor de la rentabilidad y podemos ver cuáles periodos han tenido una rentabilidad superior.Todo esto sin necesidad de reejecutar la celda cada vez que modificamos el filtro. <br>

### 2.3 Grafico con interact

In [6]:
@interact
def grafico(
    bins=(1,50,5),
    grid= True ,
    color=['blue', 'red','gray'],
    ):
    df.loc[:,'LogRet'].plot(kind='hist', 
                            bins= bins, 
                            color=color, 
                            grid=grid)

interactive(children=(IntSlider(value=21, description='bins', max=50, min=1, step=5), Checkbox(value=True, des…

### 2.4. Otra forma de utilizar interact


In [7]:
# Funcion
def filtro1(Columna, x):
    return (df.loc[df[Columna]>x])

In [8]:
# Acotamos la seleccion de valores
interact( filtro1, 
         x=widgets.FloatSlider(
             min= -0.25, 
             max=0.16, 
             step= 0.1, 
             value=0, 
             description = 'Mayor a'), 
         Columna =['LogRet']);

interactive(children=(Dropdown(description='Columna', options=('LogRet',), value='LogRet'), FloatSlider(value=…

### 2.5 Desplegable
Podemos hacer otro ejemplo donde si en vez de tener un deslizador tenemos un desplegable para filtrar con los años o los meses que necesitamos analizar. Para esto puedo sacar una lista de todos los años y todos los meses presentes en el df

In [9]:
#primero saco una lista con los años y meses presentes en el dataframe
ALL = 'ALL'
def lista_valores(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

In [10]:
lista_year = lista_valores(df.Year)
lista_month = lista_valores(df.Month)

In [11]:
#funcion
def filtro2(year, month):
    return(df.loc[(df['Year'] == year) & (df['Month'] == month)])

In [12]:
interact (filtro2, 
          year = lista_year,
          month = lista_month);

interactive(children=(Dropdown(description='year', options=('ALL', 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1…

En este filtro por desplegable hay un problema porque el dataframe no responde bien con la opción o. Por ejemplo, da error si busco todos los meses de un mismo año, o todos los años para un mismo mes. <br>
Para solucionarlo tenemos que entrar el los valores del output del widget y hacer una función que interactúe con todas las posibles selecciones.

## 3. Crear nuestros propios widgets

Si queremos que las opciones de un widget dependan del valor de otro, utilizamos la función de observación. Para esto tenemos que crear primero nuestros propios widgets

In [13]:
from IPython.display import display

In [14]:
output = widgets.Output()
outout_plot = widgets.Output()

#creo desplegable

dropdown_year = widgets.Dropdown(options = lista_year, description = 'Year')
dropdown_month = widgets.Dropdown(options = lista_month, description = 'Month')

#funcion filtrar df & plot

def filtro (Year, Month):
    output.clear_output()
    outout_plot.clear_output()
    
    if (Year == ALL) & (Month == ALL):
        filtro = df
    elif (Year == ALL):
        filtro = df.loc[(df['Month'] == Month)]
    elif (Month == ALL):
        filtro = df.loc[(df['Year']==Year)]
    else:
        filtro = df.loc[(df['Year']== Year) & (df['Month']== Month)]
        
    with output:
        display(filtro)
        
    with outout_plot:
        plt.figure (figsize= (10,12))
        plt.subplot(211)      
        filtro['LogRet'].plot(kind = 'hist', bins = 12, grid=True)
        avg = filtro['LogRet'].mean()
        plt.axvline(x=avg, color='r', linestyle ='--', label= 'Media')
        plt.legend()
        plt.title('Histograma de frecuencias', loc='right')
        plt.xlabel('Rentabilidad')
        
        plt.subplot(212)
        filtro['LogRet'].plot.bar()
        plt.title('Diagrama de barras', loc='right')
        plt.ylabel('Rentabilidad')
        plt.axhline(y=avg, color='r', linestyle ='--', label= 'Media')
        plt.legend()
        plt.show()
        
def dropdown_year_eventhandler (change):
    filtro(change.new, dropdown_month.value)
    
def dropdown_month_eventhander(change):
    filtro(dropdown_year.value, change.new)
    
dropdown_year.observe(dropdown_year_eventhandler, names ='value')
dropdown_month.observe (dropdown_month_eventhander, names = 'value')

# widgets layout
tab  = widgets.Tab([output, outout_plot])
tab.set_title(0,'Datos')
tab.set_title(1,'Gráficos')
dashboard = widgets.VBox([dropdown_year,dropdown_month, tab])
display(dashboard);